<a href="https://colab.research.google.com/github/aliash98/Search2Vec-BScThesis/blob/main/Implementation%20on%20AOL%20Dataset/AOL_Search2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import itertools
from datetime import datetime
import pandas as pd
import numpy as np
import json
import re
import warnings
import gc 
from tqdm import tqdm, trange

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/train.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567047 entries, 0 to 567046
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AnonID     567047 non-null  int64  
 1   Query      567047 non-null  object 
 2   QueryTime  567047 non-null  object 
 3   ItemRank   300763 non-null  float64
 4   ClickURL   300763 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 21.6+ MB


In [ ]:
data['Query'].value_counts().head(50)

In [ ]:
# data[data['ClickURL'].isnull()]
data.dropna(subset = ["ClickURL"], inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300763 entries, 1 to 567045
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AnonID     300763 non-null  int64  
 1   Query      300763 non-null  object 
 2   QueryTime  300763 non-null  object 
 3   ItemRank   300763 non-null  float64
 4   ClickURL   300763 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 13.8+ MB


In [ ]:
# print(data['ClickURL'].value_counts()[3027])

# Keeping the URLs repeated more than 40 times. -----> 3943 URLs
print((data['ClickURL'].value_counts()==10).argmax())


data = data[data.ClickURL.isin(data['ClickURL'].value_counts().head(2915).index)]
data.info()

2915
<class 'pandas.core.frame.DataFrame'>
Int64Index: 247952 entries, 1 to 567045
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AnonID     247952 non-null  int64  
 1   Query      247952 non-null  object 
 2   QueryTime  247952 non-null  object 
 3   ItemRank   247952 non-null  float64
 4   ClickURL   247952 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 11.4+ MB


In [ ]:
# Not necessary, has been done in the preparation

# print(data['Query'].value_counts()[3027])
# Keeping the Queries repeated more than 15 times ----> it reduced the number of urls to 3009

# print((data['Query'].value_counts()==10).argmax())
# data = data[data.Query.isin(data['Query'].value_counts().head(15286).index)]
# data.info()

15286
<class 'pandas.core.frame.DataFrame'>
Int64Index: 462534 entries, 5 to 2720986
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AnonID     462534 non-null  int64  
 1   Query      462534 non-null  object 
 2   QueryTime  462534 non-null  object 
 3   ItemRank   462534 non-null  float64
 4   ClickURL   462534 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 21.2+ MB


In [ ]:
# Constructing tokens
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0
# Importing queries to vocab
for query in data.Query:
  if query not in vocab:
    vocab[query] = index
    index += 1

print(index)

# Importing products to vocab

for url in data.ClickURL:
  if url + '-url' not in vocab:
    vocab[url + '-url'] = index
    index += 1


vocab_size = len(vocab)
print(vocab.keys())
print(index)

4981
dict_keys(['<pad>', 'emma watson', 'citation machine', 'xm radio', 'kbb.com', 'mapquest', 'pogogames', 'verizon', 'ebay', 'passport', 'google.com', 'mass lottery', 'google', 'wetcircle.com', 'verizon.net', 'hotmail.com', 'makehimpay.net', 'daytona bike week', 'free stuff', 'phone numbers', 'people search', 'jc penny', 'bed bath and beyond', 'jetblue.com', 'googlecom', 'nydailynews.com', 'yahoo', 'dictionary', 'ticket master', 'bellsouth', 'sears', 'cinelli', 'target', 'greeting cards', 'ymca', "kohl's", 'msnbc', 'dogpile', 'debra lafave', 'rotten.com', 'free fonts', 'playboy', 'wwe.com', 'ask jeeves', 'remax.com', 'pregnancy', 'bare minerals', 'prudential', 'google. com', 'toys r us', 'paragonfcu', 'area codes', 'ikea', 'yellow pages', 'bank of america.com', 'www.bank of america.com', 'www bankofamerica.com', 'tmobile', 'nickelodeon', 'photobucket.com', 'imageshack', 'meijer', 'white pages', 'www.socialsecurity.gov', 'fandango', 'discovercard.com', 'msn.com', 'yahoo.com', 'tax for

In [ ]:
json.dump( vocab, open( r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/vocab.json', 'w' ) )

In [ ]:
import random

default = np.zeros(5, dtype=int)
default[0] = 1
# print(default)
targets = []
contexts = []
labels = []
temp = np.array(5)
for ind in data.index:
    targets.append(vocab[data['Query'][ind]])
    temp = np.zeros(5, dtype=int)

    temp[0] = vocab[data['ClickURL'][ind] + '-url']

    # getting 4 random negative samples
    for i in range(1, 5):
      temp[i] = random.randint(0, vocab_size - 1)
    contexts.append(temp)
    labels.append(default)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

gc.collect()
print(targets[:5])
print(contexts[:10,:5])
print(labels[:10,:5])

[1 2 2 2 2]
[[4981 4847  569 6974 7470]
 [4982 6807  977 3121 1686]
 [4982 4555 6404 3926 4994]
 [4982 1886 6924 6826 7291]
 [4982 4039 2183 4266  516]
 [4982 4966 1837 1778 1787]
 [4983 5801 1677 1601 2996]
 [4984 7538 3491 2720 5089]
 [4984   69 3009 7727 4631]
 [4985 2036 4002 3535 6024]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]


In [ ]:
print(contexts.shape)
print(type(contexts))

(288355, 5)
<class 'numpy.ndarray'>


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    
    # *** should we change right here?
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1,
                                       name="w2v_embedding2")

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [ ]:
embedding_dim = 300
num_ns = 4
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
gc.collect()
word2vec.fit(dataset, epochs=1, callbacks=[tensorboard_callback])

242/242 [==============================] - 13s 50ms/step - loss: 0.8787 - accuracy: 0.9228


In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
weights2 = word2vec.get_layer('w2v_embedding2').get_weights()[0]
average_weights = ( weights + weights2 ) / 2.0
print(average_weights.shape)
print(str(weights[0, 0]) + "  " + str(weights2[0, 0]) + "  " + str(average_weights[0, 0])) 

# vocab = vectorize_layer.get_vocabulary()

(7896, 300)
-0.047716867  0.01842857  -0.014644149


In [ ]:
pd.DataFrame(average_weights).to_csv(r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/weights.tsv', sep="\t", index=False, header=None)